In [16]:
# encoding: utf-8
# import tensorflow as tf
import datetime
import pandas as pd
import akshare as ak

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

def get_trade_rank(market = 'SHF', date = None):
    if date is None:
        date = get_target_date(-1, "%Y-%m-%d")
    if market == 'SHF':
        return ak.get_shfe_rank_table(date)
    if market == 'DCE':
        return ak.get_dce_rank_table(date)
    if market == 'CZC':
        return ak.get_czce_rank_table(date)
    if market == "CFE":
        return ak.get_cffex_rank_table(date)
    return None, '不支持的市场类型'
def handle():
    """
    handle data
    - 去除逗号，
    - 转化为浮点数类型
    """
    markets = ['CZC', 'SHF', 'CFE', 'DCE']  # , 'CZC', 'SHF','CFE','DCE'
    df = pd.DataFrame()
    for market in markets:
        begin = datetime.date(2020, 10, 19)
        end = datetime.date(2020, 10, 19)
        # begin = datetime.date.today()
        # end = begin
        for i in range((end - begin).days + 1):
            day = begin + datetime.timedelta(days=i)
            days = day.strftime('%Y%m%d')
            try:
                df1 = get_trade_rank(market, date=days)
                for key, value in df1.items():
                    value['date'] = days
                    if market == 'CZC':
                        value = value[value['symbol'] == value['variety']]
                        value = value.applymap(lambda x: x.replace(',', '').replace("-", ""))
                        df = df.append(value)
                    if market == 'SHF':
                        value = value[-value['rank'].isin([999])]
                        df = df.append(value)
                    if market == 'DCE':
                        value = value[-value['rank'].isin([999])]
                        # value = value.applymap(lambda x: x.replace(',', '').replace("-", ""))
                        df = df.append(value)
                    if market == 'CFE':
                        value = value[-value['rank'].isin([999])]
                        df = df.append(value)
#                     print(df.tail())
            except:
                print(days, market, '数据异常')
                continue
    df = df.apply(pd.to_numeric, errors="ignore")
    #净持仓
    long = df.groupby(['date', 'variety', 'long_party_name'])[
        ['long_open_interest']].sum()
    short = df.groupby(['date', 'variety', 'short_party_name'])[
        ['short_open_interest']].sum()
    # # 合并
    frames = [long, short]
    position = pd.concat(frames, axis=1, sort=True).fillna(0).reset_index()
    # 字段更名
    position = position.rename(columns={'level_0': 'date', 'level_1': 'variety', 'level_2': 'BrokerID'})
    position['net'] = position.apply(lambda x: x['long_open_interest'] - x['short_open_interest'], axis=1)
    party_names = ['永安期货', '海通期货', '中信期货']
    df = pd.DataFrame()
    for i in party_names:
        try:
            mem = position[position['BrokerID'] == i]
            df1 = pd.DataFrame(mem)
            df = df.append(df1)
        except:
            print('?')
            continue
    two_level_index_series = df.set_index(['date','variety', 'BrokerID'])['net']
    net_df = two_level_index_series.unstack()
    net_df['合计'] = net_df.apply(lambda x: x.sum(), axis=1)
    net_df = net_df.rename_axis(columns=None).reset_index()
    net_df = net_df[['date','variety', '永安期货', '海通期货', '中信期货', '合计']]
    
#     net_df.to_excel(path, index=False)
    net_df
    return net_df

if __name__ == '__main__':
    path='d:/PyFile/np.xlsx'
#     handle(path).net_df
    display(handle())

,date,variety,永安期货,海通期货,中信期货,合计
0,20201019,A,-5856.0,2150.0,3842.0,136.0
1,20201019,AG,11654.0,17629.0,18061.0,47344.0
2,20201019,AL,6131.0,-13768.0,-2093.0,-9730.0
3,20201019,AP,-6673.0,501.0,824.0,-5348.0
4,20201019,AU,5741.0,6397.0,11013.0,23151.0
5,20201019,BU,-44227.0,7908.0,-19416.0,-55735.0
6,20201019,C,84530.0,21981.0,-84945.0,21566.0
7,20201019,CF,-8518.0,22319.0,-4996.0,8805.0
8,20201019,CJ,-946.0,544.0,-345.0,-747.0
9,20201019,CS,4406.0,4032.0,798.0,9236.0
